In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, f1_score
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
import numpy as np
import random

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed()

In [2]:
# 模块 2：定义模型

# 定义单模态模型（适用于 EEG 或 fMRI）
class SingleModalModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, output_size=1):
        super(SingleModalModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        return out

# 定义多模态模型（适用于 EEG+fMRI）
class MultiModalModel(nn.Module):
    def __init__(self, input_size_eeg, input_size_fmri, hidden_size=256, output_size=1):
        super(MultiModalModel, self).__init__()
        self.fc_eeg = nn.Linear(input_size_eeg, hidden_size)
        self.fc_fmri = nn.Linear(input_size_fmri, hidden_size)
        self.relu = nn.ReLU()
        self.fc_out = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, eeg, fmri):
        out_eeg = self.relu(self.fc_eeg(eeg))
        out_fmri = self.relu(self.fc_fmri(fmri))
        combined = torch.cat((out_eeg, out_fmri), dim=1)
        out = self.fc_out(combined)
        return out


In [3]:
# 模块 3：定义合成数据集

# 生成合成数据集
class SyntheticDataset(Dataset):
    def __init__(self, num_samples=1000, input_size_eeg=64, input_size_fmri=448, task='cognitive_load'):
        """
        任务：
            - cognitive_load
            - stress
        """
        super(SyntheticDataset, self).__init__()
        self.num_samples = num_samples
        self.input_size_eeg = input_size_eeg
        self.input_size_fmri = input_size_fmri
        self.task = task

        # 根据任务设置标签相关性
        if self.task == 'cognitive_load':
            # 认知负荷检测，标签为1时，EEG和fMRI有更高的分离度
            self.eeg_data = self.generate_data(input_size_eeg, class_sep=2.0)
            self.fmri_data = self.generate_data(input_size_fmri, class_sep=2.0)
        elif self.task == 'stress':
            # 压力检测，标签为1时，EEG和fMRI有不同的更高分离度
            self.eeg_data = self.generate_data(input_size_eeg, class_sep=2.2)
            self.fmri_data = self.generate_data(input_size_fmri, class_sep=2.2)
        else:
            raise ValueError("Unsupported task")

        self.labels = torch.randint(0, 2, (num_samples, 1)).float()

    def generate_data(self, input_size, class_sep=2.0):
        # 对于标签为1和0的数据，生成不同分布的数据
        class_1 = torch.randn(self.num_samples // 2, input_size) + class_sep
        class_0 = torch.randn(self.num_samples - self.num_samples // 2, input_size)
        data = torch.cat((class_1, class_0), dim=0)
        return data

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.eeg_data[idx], self.fmri_data[idx], self.labels[idx]


In [4]:
# 模块 4：定义训练和验证函数

# 训练模型（单模态）
def train_single_modal(model, criterion, optimizer, dataloader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for data_batch, labels_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(data_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * data_batch.size(0)
        epoch_loss /= len(dataloader.dataset)
        print(f"[Single-Modal] Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# 训练模型（多模态）
def train_multi_modal(model, criterion, optimizer, dataloader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for eeg_batch, fmri_batch, labels_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(eeg_batch, fmri_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * eeg_batch.size(0)
        epoch_loss /= len(dataloader.dataset)
        print(f"[Multi-Modal] Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# 验证模型（单模态）
def validate_single_modal(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data_batch, labels_batch in dataloader:
            outputs = model(data_batch)
            preds = torch.sigmoid(outputs) > 0.5
            all_labels.append(labels_batch.cpu())
            all_preds.append(preds.cpu())
    all_labels = torch.cat(all_labels).numpy()
    all_preds = torch.cat(all_preds).numpy()
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return accuracy, precision, f1

# 验证模型（多模态）
def validate_multi_modal(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for eeg_batch, fmri_batch, labels_batch in dataloader:
            outputs = model(eeg_batch, fmri_batch)
            preds = torch.sigmoid(outputs) > 0.5
            all_labels.append(labels_batch.cpu())
            all_preds.append(preds.cpu())
    all_labels = torch.cat(all_labels).numpy()
    all_preds = torch.cat(all_preds).numpy()
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return accuracy, precision, f1


In [5]:
# 模块 5：定义评估和性能表格生成函数

# 评估模型并收集性能指标
def evaluate_model(task, modality, num_runs=10, batch_size=32, num_samples=1000):
    accuracies = []
    precisions = []
    f1_scores = []

    for run in range(num_runs):
        print(f"Run {run+1}/{num_runs} for Task: {task}, Modality: {modality}")

        # 创建数据集和数据加载器
        dataset = SyntheticDataset(num_samples=num_samples, task=task)
        
        if modality == 'EEG':
            # 仅使用 EEG 数据
            eeg_data = dataset.eeg_data
            labels = dataset.labels
            eeg_dataset = TensorDataset(eeg_data, labels)
            eeg_loader = DataLoader(eeg_dataset, batch_size=batch_size, shuffle=True)

            # 初始化单模态模型
            model = SingleModalModel(input_size=64)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            # 训练模型
            train_single_modal(model, criterion, optimizer, eeg_loader, num_epochs=10)

            # 验证模型
            accuracy, precision, f1 = validate_single_modal(model, eeg_loader)
            accuracies.append(accuracy)
            precisions.append(precision)
            f1_scores.append(f1)

        elif modality == 'fMRI':
            # 仅使用 fMRI 数据
            fmri_data = dataset.fmri_data
            labels = dataset.labels
            fmri_dataset = TensorDataset(fmri_data, labels)
            fmri_loader = DataLoader(fmri_dataset, batch_size=batch_size, shuffle=True)

            # 初始化单模态模型
            model = SingleModalModel(input_size=448)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            # 训练模型
            train_single_modal(model, criterion, optimizer, fmri_loader, num_epochs=10)

            # 验证模型
            accuracy, precision, f1 = validate_single_modal(model, fmri_loader)
            accuracies.append(accuracy)
            precisions.append(precision)
            f1_scores.append(f1)

        elif modality == 'EEG+fMRI':
            # 使用 EEG 和 fMRI 数据
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            # 初始化多模态模型
            model = MultiModalModel(input_size_eeg=64, input_size_fmri=448)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            # 训练模型
            train_multi_modal(model, criterion, optimizer, dataloader, num_epochs=10)

            # 验证模型
            accuracy, precision, f1 = validate_multi_modal(model, dataloader)
            accuracies.append(accuracy)
            precisions.append(precision)
            f1_scores.append(f1)

        else:
            raise ValueError("Unsupported modality")

    # 计算平均值和标准差
    accuracy_mean = np.mean(accuracies) * 100
    accuracy_std = np.std(accuracies) * 100
    precision_mean = np.mean(precisions) * 100
    precision_std = np.std(precisions) * 100
    f1_mean = np.mean(f1_scores) * 100
    f1_std = np.std(f1_scores) * 100

    return accuracy_mean, accuracy_std, precision_mean, precision_std, f1_mean, f1_std

# 构建最终表格
def build_performance_table():
    tasks = ['cognitive_load', 'stress']
    modalities = ['EEG', 'fMRI', 'EEG+fMRI']
    performance_results = []

    for task in tasks:
        for modality in modalities:
            accuracy_mean, accuracy_std, precision_mean, precision_std, f1_mean, f1_std = evaluate_model(
                task=task,
                modality=modality,
                num_runs=10,
                batch_size=32,
                num_samples=1000
            )
            task_name = "Cognitive Load Detection" if task == 'cognitive_load' else "Stress Detection"
            if modality == 'EEG':
                modality_name = "EEG (Single Modality)"
            elif modality == 'fMRI':
                modality_name = "fMRI (Single Modality)"
            else:
                modality_name = "EEG+fMRI (Multi-Modality)"
            performance_results.append({
                "Task": task_name,
                "Modality": modality_name,
                "Accuracy (%)": f"{accuracy_mean:.1f}",
                "Precision (%)": f"{precision_mean:.1f}",
                "F1-Score (%)": f"{f1_mean:.1f}"
            })

    # 将结果展示为表格
    df = pd.DataFrame(performance_results)
    print("\nPerformance of Cognitive Load and Stress Detection:")
    print(df)


In [6]:
# 模块 6：运行主流程

# 运行并生成表格
if __name__ == "__main__":
    build_performance_table()


Run 1/10 for Task: cognitive_load, Modality: EEG
[Single-Modal] Epoch 1/10, Loss: 0.7019
[Single-Modal] Epoch 2/10, Loss: 0.6775
[Single-Modal] Epoch 3/10, Loss: 0.6856
[Single-Modal] Epoch 4/10, Loss: 0.6498
[Single-Modal] Epoch 5/10, Loss: 0.6328
[Single-Modal] Epoch 6/10, Loss: 0.6135
[Single-Modal] Epoch 7/10, Loss: 0.5988
[Single-Modal] Epoch 8/10, Loss: 0.5950
[Single-Modal] Epoch 9/10, Loss: 0.5663
[Single-Modal] Epoch 10/10, Loss: 0.5501
Run 2/10 for Task: cognitive_load, Modality: EEG
[Single-Modal] Epoch 1/10, Loss: 0.7151
[Single-Modal] Epoch 2/10, Loss: 0.6775
[Single-Modal] Epoch 3/10, Loss: 0.6687
[Single-Modal] Epoch 4/10, Loss: 0.6479
[Single-Modal] Epoch 5/10, Loss: 0.6421
[Single-Modal] Epoch 6/10, Loss: 0.6097
[Single-Modal] Epoch 7/10, Loss: 0.5958
[Single-Modal] Epoch 8/10, Loss: 0.5771
[Single-Modal] Epoch 9/10, Loss: 0.5559
[Single-Modal] Epoch 10/10, Loss: 0.5398
Run 3/10 for Task: cognitive_load, Modality: EEG
[Single-Modal] Epoch 1/10, Loss: 0.7050
[Single-Mod